In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from tensorflow.keras.layers import MaxPooling2D, Input, Concatenate, Dense, Dropout, Activation, Flatten, Conv2D, Conv1D, GlobalAveragePooling1D, GlobalAveragePooling2D, BatchNormalization, Flatten, GRU
from tensorflow.keras import Model
from tensorflow.keras.utils import plot_model

In [ ]:
# Compte rendu de classification
def cpt_mal_classes(y_test_func, result_func):
    nb_func = 0
    for i in range(len(y_test_func)):
        if y_test_func[i] != result_func[i]:
            nb_func += 1
    print (f'Taille des données {len(y_test_func)} mal classés {nb_func}\n')

# Plot confusion matrix
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / np.sum(cm).astype('float')
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

# Display image in actual size
def display_image_in_actual_size(im_path):

    dpi = 80
    im_data = plt.imread(im_path)
    height, width, depth = im_data.shape

    # What size does the figure need to be in inches to fit the image?
    figsize = width / float(dpi), height / float(dpi)

    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])

    # Hide spines, ticks, etc.
    ax.axis('off')

    # Display the image.
    ax.imshow(im_data, cmap='gray')

    plt.show()

In [ ]:
Path('Output').mkdir(exist_ok=True, parents=True)

## Recharger les données d'entraînement, de validation et test en fichier numpy

In [ ]:
# Recharger les données après avoir vidé la mémoire
train_X = np.load('Data/train_valid_test/train_X.npy')
train_y = np.load('Data/train_valid_test/train_y.npy')

valid_X = np.load('Data/train_valid_test/valid_X.npy')
valid_y = np.load('Data/train_valid_test/valid_y.npy')
valid_id = np.load('Data/train_valid_test/valid_id.npy')

test_X = np.load('Data/train_valid_test/test_X.npy')
test_id = np.load('Data/train_valid_test/test_id.npy')

train_X1 = train_X[:,2,2,:,:]
valid_X1 = valid_X[:,2,2,:,:]
test_X1 = test_X[:,2,2,:,:]

train_X2 = train_X[:,:,:,:].reshape(train_X.shape[0],20,20,-1)
valid_X2 = valid_X[:,:,:,:].reshape(valid_X.shape[0],20,20,-1)
test_X2 = test_X[:,:,:,:].reshape(test_X.shape[0],20,20,-1)

## Encoder les labels entre 0 et 4 de sorte à matcher les prédictions des réseaux de neurones

In [ ]:
encoder = LabelEncoder()

encoder.fit(train_y)
train_y_enc = encoder.transform(train_y)
valid_y_enc = encoder.transform(valid_y)

In [ ]:
# Inputs 
cnn1d_input = Input(shape=(8, 4),name='cnn1d_input')
cnn2d_input = Input(shape=(20, 20, 2),name='cnn2d_input')

# CNN1D branch
cnn1d_layer1 = Conv1D(filters=32, kernel_size=3, activation='relu')(cnn1d_input)
cnn1d_layer1 = BatchNormalization()(cnn1d_layer1)

cnn1d_layer2 = Conv1D(filters=64, kernel_size=3, activation='relu')(cnn1d_layer1)
cnn1d_layer2 = BatchNormalization()(cnn1d_layer2)
cnn1d_layer2 = Dropout(0.5)(cnn1d_layer2)

cnn1d_layer3 = GlobalAveragePooling1D()(cnn1d_layer2)

cnn1d_out = Dense(5, activation='softmax', name='cnn1d')(cnn1d_layer3)

cnn1d_branch = Model(inputs=cnn1d_input, outputs=[cnn1d_layer3, cnn1d_out], name="cnn1d")

# CNN2D branch
cnn2d_layer1 = Conv2D(filters=32,kernel_size=(5,5),activation='relu')(cnn2d_input)
cnn2d_layer1 = MaxPooling2D(pool_size=(2, 2))(cnn2d_layer1)

cnn2d_layer2 = Conv2D(filters=64,kernel_size=(3,3),activation='relu')(cnn2d_layer1)

cnn2d_layer3 = Flatten()(cnn2d_layer2)

cnn2d_out = Dense(5,activation='softmax',name='cnn2d')(cnn2d_layer3)

cnn2d_branch = Model(inputs=cnn2d_input,outputs=[cnn2d_layer3,cnn2d_out],name="cnn2d")

# Concaténation des sorties des branches
concat = Concatenate()([cnn1d_branch.output[0],cnn2d_branch.output[0]])

# Un niveau Dense pour la classification
# fc = Dense (128,activation='relu')(concat)
# fc = Dropout (0.5)(fc)
out_layer = Dense(5,activation='softmax',name='main')(concat)

model = Model(inputs=[cnn1d_branch.input, cnn2d_branch.input], outputs=[out_layer, cnn1d_branch.output[1], cnn2d_branch.output[1]], name="cnn1d_cnn2d")

model.summary()

plot_model(model, 'Output/cnn1d_cnn2d.png', show_shapes=True)

display_image_in_actual_size('Output/cnn1d_cnn2d.png')

In [ ]:
OPTIMIZER = tf.keras.optimizers.Adam(0.0001)
BATCH_SIZE = 256
EPOCHS = 25
CALLBACKS = [tf.keras.callbacks.ModelCheckpoint(
              'Model/model',
              verbose=1, # niveau de log
              monitor='val_main_acc', # nom de la métrique à surveiller
              save_best_only=True, # sauver uniquement le meilleur modèle
              save_weights_only=True)] # sauver uniquement les poids

model.compile(optimizer=OPTIMIZER, loss=tf.keras.losses.SparseCategoricalCrossentropy(), loss_weights=[1, .5, .5], metrics=tf.keras.metrics.SparseCategoricalAccuracy(name='acc'))

history = model.fit([train_X1, train_X2], train_y_enc, validation_data=([valid_X1, valid_X2], valid_y_enc), batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=CALLBACKS)

In [ ]:
model.save('Model/Best_CNN1D_CNN2D')

model_loaded1 = model.load_weights('Model/model')
model_loaded2 = keras.models.load_model('Model/Best_CNN1D_CNN2D')

In [ ]:
# summarize history for accuracy
plt.plot(history.history['main_acc'])
plt.plot(history.history['val_main_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['accuracy', 'val_accuracy'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss'], loc='upper left')
plt.show()

In [ ]:
score = model.evaluate([valid_X1, valid_X2], valid_y_enc, batch_size=256)

print(f'Loss : {score[1]:.2f}')
print(f'Accuracy : {score[4]:.2f}')

### **Prédiction des classes sur le jeu de validation et évaluation en aggrégeant au niveau objet**

In [ ]:
# Récupérer les probabilités prédites sur le jeu de validation
valid_prob, _, _ = model.predict([valid_X1, valid_X2], batch_size=256)

# Retourner la classe correspondant à la probabilité la plus haute
valid_pred = np.argmax(valid_prob, axis=1) # axe 1 car ceci concerne chaque ligne

# Je réencode les prédictions entre 1 et 5
valid_pred_enc = encoder.inverse_transform(valid_pred)

# Aggrégation au niveau objet
out_pred = []
unique_id = np.unique(valid_id)
for ID in unique_id :
    # Récupérer les prédictions des pixels appartenant au même objet
    pred = valid_pred_enc[np.where(valid_id==ID)]
    y_true = valid_y[np.where(valid_id==ID)]
    # Prendre la valeur majoritaire des prédictions sur les pixels
    out_pred.append([ np.bincount(y_true).argmax(), np.bincount(pred).argmax()]) #(Vérité terrain,Prédiction majoritaire)
out_pred = np.vstack(out_pred)
y_true = out_pred[:,0]
y_pred = out_pred[:,1]

In [ ]:
cpt_mal_classes(y_true, y_pred)
print(f'F1 score : {f1_score(y_true,y_pred,average="weighted"):.2f}\n')
print(f'Accuracy : {accuracy_score(y_pred, y_true):.2f}\n')
print(f'Matrice de confusion :\n{confusion_matrix(y_true, y_pred)}\n')
print(f'Classification report :\n{classification_report(y_true, y_pred)}')

In [ ]:
plot_confusion_matrix(confusion_matrix(y_true, y_pred), None)

### **Prédire sur le jeu test et Préparer une soumission**

In [ ]:
# Récupérer les probabilités prédites sur le jeu test
test_prob, _, _ = model.predict([test_X1, test_X2], batch_size=256)

# Retourner la classe correspondant à la probabilité la plus haute
test_pred = np.argmax(test_prob, axis=1) # axe 1 car ceci concerne chaque ligne

# Je réencode les prédictions entre 1 et 5
test_pred_enc = encoder.inverse_transform(test_pred)

# Aggrégation au niveau objet
agg_pred = []
unique_id = np.unique(test_id)
for ID in unique_id :
    # Récupérer les prédictions des pixels appartenant au même objet
    pred = test_pred_enc[np.where(test_id==ID)]
    # Prendre la valeur majoritaire des prédictions sur les pixels
    agg_pred.append([ ID, np.bincount(pred).argmax()]) #(ID,Prédiction majoritaire)
agg_pred = np.vstack(agg_pred)

In [ ]:
df = pd.DataFrame({'ID':agg_pred[:, 0], 'Class':agg_pred[:, 1]})
df_test = pd.read_csv('Data/Test_id_Label.csv')

In [ ]:
cpt_mal_classes(df_test.Class, df.Class)
print(f'F1 score : {f1_score(df_test.Class,df.Class,average="weighted"):.2f}\n')
print(f'Accuracy : {accuracy_score(df.Class, df_test.Class):.2f}\n')
print(f'Matrice de confusion :\n{confusion_matrix(df_test.Class, df.Class)}\n')
print(f'Classification report :\n{classification_report(df_test.Class, df.Class)}')

In [ ]:
plot_confusion_matrix(confusion_matrix(df_test.Class, df.Class), None)